In [1]:
include("../../../julia/FNC_init.jl")

  Activating 

project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-newton-converge)

We again look at finding a solution of $x e^x=2$ near $x=1$. To apply Newton's method, we need to calculate values of both the residual function $f$ and its derivative.

In [2]:
f(x) = x * exp(x) - 2;
df_dx(x) = exp(x) * (x + 1);

We don't know the exact root, so we use `nlsolve` to determine a proxy for it.

In [3]:
using NLsolve
r = nlsolve(x -> f(x[1]), [1.0]).zero

1-element Vector{Float64}:
 0.852605502013726

We use $x_1=1$ as a starting guess and apply the iteration in a loop, storing the sequence of iterates in a vector.

In [4]:
x = [1; zeros(4)]
for k = 1:4
    x[k+1] = x[k] - f(x[k]) / df_dx(x[k])
end
x

5-element Vector{Float64}:
 1.0
 0.8678794411714423
 0.8527833734164099
 0.8526055263689221
 0.852605502013726

Here is the sequence of errors.

In [5]:
ϵ = @. x - r

5-element Vector{Float64}:
 0.14739449798627402
 0.015273939157716354
 0.00017787140268388235
 2.435519608212644e-8
 0.0

Because the error reaches machine epsilon so rapidly, we're going to use extended precision to allow us to take a few more iterations. We'll take the last iteration as the most accurate root estimate.
```{tip}
:class: dropdown
A `BigFloat` uses 256 bits of precision, rather than 53 in `Float64`. But arithmetic is done by software emulation and is much slower.
```

In [6]:
x = [BigFloat(1); zeros(7)]
for k = 1:7
    x[k+1] = x[k] - f(x[k]) / df_dx(x[k])
end
r = x[end]

0.8526055020137254913464724146953174668984533001514035087721073946525150656742605

In [7]:
ϵ = @. Float64(x[1:end-1] - r)

7-element Vector{Float64}:
 0.14739449798627452
 0.01527393915771683
 0.00017787140268443004
 2.435519656311045e-8
 4.56680051680793e-16
 1.6056572825272187e-31
 1.9848810119594387e-62

The exponents in the scientific notation definitely suggest a squaring sequence. We can check the evolution of the ratio in {eq}`quadratictest`.

In [8]:
logerr = @. log10(abs(ϵ))
ratios = [NaN; [logerr[i+1] / logerr[i] for i in 1:length(logerr)-1]]
@pt :header=["iteration", "error", "log error", "ratio"] [1:7 ϵ logerr ratios]

iteration,error,log error,ratio
1.0,0.147394,-0.831519,NaN
2.0,0.0152739,-1.81605,2.18401
3.0,0.000177871,-3.74989,2.06486
4.0,2.43552e-8,-7.61341,2.0303
5.0,4.5668e-16,-15.3404,2.01492
6.0,1.60566e-31,-30.7943,2.0074
7.0,1.98488e-62,-61.7023,2.00369


The clear convergence to 2 above constitutes good evidence of quadratic convergence.